In [ ]:
pip install gurobipy

In [17]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

#tried for smaller size as it is countering error for large size.
# Define problem parameters
num_locations = 10
num_drones = 5
drone_capacity = 10

# Generate random problem data
locations = np.random.rand(num_locations, 2) * 100
distances = {(i, j): np.sqrt((locations[i, 0] - locations[j, 0])**2 + (locations[i, 1] - locations[j, 1])**2) for i in range(num_locations) for j in range(num_locations)}
weights = {(i, j): np.random.randint(1, 11) for i in range(num_locations) for j in range(num_locations)}
satisfaction_scores = {(i, j): np.random.randint(1, 11) for i in range(num_locations) for j in range(num_locations)}

# Create a new Gurobi model
model = gp.Model("Delivery Drone Logistics")

# Define the decision variables
x = model.addVars(num_locations, num_locations, vtype=GRB.BINARY, name="x")

# Define the objective function
obj = gp.quicksum(distances[i, j] * x[i, j] for i in range(num_locations) for j in range(num_locations))
model.setObjective(obj, GRB.MINIMIZE)

# Add the constraints
for i in range(num_locations):
    model.addConstr(gp.quicksum(x[i, j] for j in range(num_locations)) == 1)
for j in range(num_locations):
    model.addConstr(gp.quicksum(x[i, j] for i in range(num_locations)) == 1)
model.addConstr(gp.quicksum(weights[i, j] * x[i, j] for i in range(num_locations) for j in range(num_locations)) <= num_drones * drone_capacity)

# Optimize the model
model.optimize()

# Print the results
print("Optimal Solution:")
for i in range(num_locations):
    for j in range(num_locations):
        if x[i, j].X > 0.5:
            print(f"Drone travels from location {i} to location {j}")
            print(f"Package weight: {weights[i, j]} kg")
            print(f"Customer satisfaction score: {satisfaction_scores[i, j]}")
            print()


Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-1355U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 21 rows, 100 columns and 300 nonzeros
Model fingerprint: 0x92206246
Variable types: 0 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+00, 9e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+01]
Found heuristic solution: objective 413.4638023
Presolve time: 0.00s
Presolved: 21 rows, 100 columns, 281 nonzeros
Variable types: 0 continuous, 100 integer (100 binary)

Root relaxation: objective 9.033645e-01, 12 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.90336    0    4  413.463